# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import googlemaps
from datetime import datetime
import citipy
from citipy import citipy

# Import API key
from api_keys import gkey

gmaps.configure(api_key=gkey)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# read in the data

city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Create a heat map that displays the humidity for every city from the part I of the homework.

# Store 'Lat' and 'Lng' into  locations
locations = city_data_df[['Lat', 'Lng']].astype(float)
locations

,Lat,Lng
0,68.05,39.51
1,-20.41,57.70
2,77.48,-69.36
3,40.71,112.04
4,-33.93,18.42
...,...,...
543,33.97,36.66
544,24.01,-104.61
545,49.65,0.71
546,51.07,132.56


In [20]:
# Create a heat map
fig = gmaps.figure(zoom_level = 1.9, center = (33.7,00))

heat_layer = gmaps.heatmap_layer(locations, weights=city_data_df['Humidity'], 
                                 dissipating=False, max_intensity=100, point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
    # A max temperature lower than 80 degrees but higher than 70.
    # Wind speed less than 10 mph.
    # Zero cloudiness.
    # Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
city_data_filtered_df = city_data_df.loc[city_data_df['Max Temp'] < 80]
city_data_filtered_df = city_data_filtered_df.loc[city_data_filtered_df['Max Temp'] > 70]
city_data_filtered_df = city_data_filtered_df.loc[city_data_filtered_df['Cloudiness'] == 0]
city_data_filtered_df = city_data_filtered_df.loc[city_data_filtered_df['Wind Speed'] < 10]
city_data_filtered_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# create the Hotel DataFrame

hotel_df = pd.DataFrame()

# add the columns needed
hotel_df['Hotel Name'] = ['Hilton', 'Days Inn']
hotel_df['City'] = city_data_filtered_df['City']
hotel_df['Country'] = city_data_filtered_df['Country']

# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

# partial url
# url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&fields=photos,formatted_address,name,opening_hours,rating&locationbias=circle:2000@37.4222339,-122.0854804"
url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&fields=photos,formatted_address,name,opening_hours,rating&locationbias=circle:"
units = "metric"
dist = 5000
#lat = -18.06
#lng = 13.84

# for city in city_data_filtered_df:
locations['Latitude'] = lat
locations['Longitude'] = lng

cities = []

locations['Locations'] = list(zip(locations['Latitude'], locations['Longitude']))

# Loop through the list of locations to get the nearby cities
for location in locations['Locations']:
    lat, lng = location
    city = citipy.nearest_city(lat, lng).city_name
    if city not in cities:
        cities.append(city)    
        
        # Build query URL
        query_url = f"{url}{dist}@{lat},{lng}&key={gkey}"
        response = requests.get(query_url).json()
        response['candidates'][0]['formatted_address']
        response['candidates'][0]['name']

hotel_df

NameError: name 'lat' is not defined

In [ ]:
# Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer on top of heat map


# Display figure
